In [17]:
import pandas as pd
from textblob import TextBlob
import os
import datetime as dt

In [18]:
symbols = []
text = []
date = []
with open("newsdata.txt", 'rb') as myfile:
    for line in myfile.readlines():
        line = line.strip("\n").split("||")
        symbols.append(line[0])
        text.append(line[1])
        date.append(line[2])
news_data = pd.DataFrame({"company": symbols, "text": text, "date":date})
news_data['date'] = pd.to_datetime(news_data['date'])
#news_data.to_csv("new_data.csv", index =False)

In [19]:
news_data.head()

company       date                                               text
0     MAC 2017-10-16  Breaking: Macerich (NYSE:MAC) Buy Rating Maint...
1     MAC 2017-10-14  Chilton Capital Management Has Trimmed Maceric...
2     MAC 2017-10-14  Chilton Capital Management Lowered Its Stake i...
3     MAC 2017-10-16  Indus Capital Partners Has Lifted By $912520 I...
4     MAC 2017-10-16  Analysts See $0.98 EPS for Macerich Co (MAC); ...

#### Develop individual-event level features

In [20]:
#some direct text feature
news_data['scandal'] = news_data['text'].map(lambda x : 1 if 'scandal' in x.lower() else 0)
#news_data['CFO'] = news_data['text'].map(lambda x : 1 if 'CFO' in x.lower() else 0) #none of these have CFO
news_data['buy'] = news_data['text'].map(lambda x : 1 if 'buy' in x.lower() else 0)
news_data['sell'] = news_data['text'].map(lambda x : 1 if 'sell' in x.lower() else 0)
news_data['decline'] = news_data['text'].map(lambda x : 1 if 'decline' in x.lower() else 0)
news_data['decline'].value_counts()
#sentiment feature
#bag-of-words

0    18016
1      148
Name: decline, dtype: int64

#### Sentiment

In [21]:
def get_polarity(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity
def get_subjectivity(text):
    blob = TextBlob(text)
    return blob.sentiment.subjectivity

In [51]:
news_data["positivity"] = news_data['text'].map(lambda x : get_polarity(x))
news_data["subjectivity"] = news_data['text'].map(lambda x : get_subjectivity(x))
news_data["financial_report_quarter"] = 0
news_data.ix[news_data['text'].str.upper().str.contains('REPORTS Q'),"financial_report_quarter"] = pd.to_datetime(news_data['date']).dt.quarter
news_data.head()

company       date                                               text  \
0     MAC 2017-10-16  Breaking: Macerich (NYSE:MAC) Buy Rating Maint...   
1     MAC 2017-10-14  Chilton Capital Management Has Trimmed Maceric...   
2     MAC 2017-10-14  Chilton Capital Management Lowered Its Stake i...   
3     MAC 2017-10-16  Indus Capital Partners Has Lifted By $912520 I...   
4     MAC 2017-10-16  Analysts See $0.98 EPS for Macerich Co (MAC); ...   

   scandal  buy  sell  decline  positivity  subjectivity  \
0        0    1     0        0         0.0           0.0   
1        0    0     0        0         0.0           0.0   
2        0    0     0        0         0.0           0.0   
3        0    0     0        0         0.0           0.0   
4        0    0     0        0         0.0           0.0   

   financial_report_quarter  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0

In [54]:
for_analysis = news_data[['company', 'date', 'scandal', 'buy', 'sell', 'decline', 'positivity', 'subjectivity','financial_report_quarter']]
for_analysis = for_analysis.groupby(['company', 'date'], as_index=False).agg(sum)
for_analysis.head()

company       date  scandal  buy  sell  decline  positivity  subjectivity  \
0     COL 2015-10-30        0    0     0        0         0.0      0.000000   
1     COL 2015-11-02        0    0     0        0         0.0      0.076923   
2     COL 2015-11-11        0    0     0        0         0.0      0.000000   
3     COL 2016-01-26        0    0     0        0         0.0      0.000000   
4     COL 2016-02-01        0    0     0        0         0.0      0.076923   

   financial_report_quarter  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0

In [55]:
folder = "nlp_by_company_fr"
os.mkdir(folder)
for company  in list(for_analysis['company']):
    data = for_analysis[for_analysis['company'] == company]
    data = data[['date', 'scandal', 'buy', 'sell', 'decline', 'positivity', 'subjectivity', 'financial_report_quarter']]
    data.to_csv(folder + "/"+ company + ".csv", index=False)

In [56]:
for_analysis.to_csv("preliminary_features.csv")

#### Develop Company-Level Features

In [57]:
functs = {"date": ['min', 'max'], "text": 'count'}
#add average sentiment, sentement std and 
news_data.groupby('company')['date'].agg(functs)

date            text
               min        max date
company                           
COL     2015-10-30 2017-10-16  121
CRM     2015-11-18 2017-10-16  122
DGX     2015-10-22 2017-10-16  115
FOX     2016-01-18 2017-10-16  153
FOXA    2016-01-18 2017-10-16  153
FTI     2017-10-03 2017-10-15   15
JWN     2015-11-12 2017-10-16  125
KORS    2016-06-01 2017-10-16  115
LUV     2015-11-04 2017-10-16  161
M       2015-11-11 2017-10-16  138
MA      2015-10-27 2017-10-16  135
MAA     2015-10-28 2017-10-16  102
MAC     2015-10-07 2017-10-16   95
MAR     2015-10-28 2017-10-16  123
MAS     2015-12-15 2017-10-16  100
MAT     2016-01-27 2017-10-16  114
MCD     2015-10-27 2017-10-16  153
MCHP    2015-11-04 2017-10-16  100
MCK     2015-10-29 2017-10-16  127
MCO     2015-10-21 2017-10-16  114
MDLZ    2015-10-28 2017-10-16  139
MDT     2017-10-02 2017-10-16   95
MET     2015-10-27 2017-10-16  123
MGM     2015-10-29 2017-10-16  124
MHK     2015-11-05 2017-10-16   91
MKC     2015-11-24 2017-10-16  108
MLM     2015-11-13 2017-10-16   92
MMC     2015-11-04 2017-10-16  112
MNST    2017-10-02 2017-10-16   86
MON     2015-10-20 2017-10-16  172
...            ...        ...  ...
SRE     2015-12-02 2017-10-16  134
STI     2015-11-10 2017-10-16  119
STT     2015-10-23 2017-10-16  112
STX     2015-10-21 2017-10-16  102
SWK     2015-10-22 2017-10-16  107
SWKS    2015-10-30 2017-10-16   97
SYF     2016-03-01 2017-10-16  104
SYK     2015-10-26 2017-10-16  119
SYMC    2015-12-21 2017-10-16  114
SYY     2015-11-18 2017-10-16  103
TAP     2015-11-05 2017-10-16  114
TDG     2015-11-19 2017-10-16  111
TEL     2015-12-02 2017-10-16  106
TGT     2015-10-27 2017-10-16  141
TIF     2016-01-19 2017-10-16  116
TJX     2016-05-17 2017-10-16  109
TMK     2015-10-26 2017-10-16   91
TMO     2015-11-12 2017-10-16  128
TRIP    2016-06-07 2017-10-16  107
TROW    2016-02-17 2017-10-16  110
TRV     2016-06-17 2017-10-16   95
TSCO    2015-10-21 2017-10-16  100
TSN     2015-11-20 2017-10-16  117
TSS     2015-10-21 2017-10-16  109
TWX     2015-11-17 2017-10-16  158
TXN     2016-01-21 2017-10-16   93
TXT     2015-10-21 2017-10-16  123
UDR     2015-10-26 2017-10-16  120
ULTA    2015-12-03 2017-10-16  101
USB     2017-10-02 2017-10-16   27

[160 rows x 3 columns]